# 14.4.3-14.4.7_CNNの代表的なアーキテクチャ

CNNの代表的なアーキテクチャを紹介する。<br>
なお、実装は14.5や14.6で行うので今回は各アーキテクチャのアイデアを紹介する。

■今回紹介するアーキテクチャ一覧<br>
| アーキテクチャ             | 登場年 | ILSVRC top-5 error rate | 備考                                         | 
| -------------------------- | ------ | ----------------------- | -------------------------------------------- | 
| VGG                        | 2014   | 6.8%                    |                                              | 
| GoogLeNet (Inception v1)   | 2014   | 6.7%                    |                                              | 
| ResNet                     | 2015   | 3.57%                   |                                              | 
| Xception (Inception派生型) | 2016   | -                       | Googleの検証でInception v3の性能を引き離した | 
| SENet                      | 2017   | 2.25%                   |                                              | 

## VGG

### 特徴
* AlexNetを発展させてより深いモデルを実現
* 畳み込み層を重ねている
    - そのメリットとしては、パラメータ数を減らしつつ同じ表現力を保証できる。<br>
    例えばカーネルサイズ5×5の畳み込みは3×3の2層で表現できる。<br>
    この場合パラメータ数を比較すると5×5＝25に対して3×3×2=18なので18/25=0.72で約3割のパラメータを削減できる。<br>
    ![](./fig/vgg_replace_mini_network.png)<br>
    図は https://ai-kenkyujo.com/artificial-intelligence/ai-architecture-02/ より

### アーキテクチャ

* Conv層を2~3層連続させた後にMaxPoolingを挟む
* Pooling層のストライド幅はすべて2
* 活性化関数はReLU

* 図はVGGアーキテクチャの一覧（左→右に深くなる。各層の数字は"フィルターサイズ-フィルター数"）<br>
![](./fig/vgg.png)<br>
図は https://qiita.com/ttomomasa/items/b673a1e0b42a2a14a9d2 より（※元論文のキャプチャ）

* 論文によると、検証ではDとEが性能が良かったらしい。

## GoogLeNet（Inception v1）

### 特徴

* AlexNet、VGGに比べてはるかに効率的にパラメータを使えるようなったモデル
* インセプションモジュールと呼ばれるサブネットワークを組込んでいる

### アーキテクチャ

* アーキテクチャは以下： <br>
    ![](./fig/googLeNet.png)<br>
※層内の数字は（アウトプットチャンネル数, フィルターサイズ+ストライド幅）。<br>インセプションモジュールの数字は後述のインセプションモジュール図の各ノードのアウトプットチャンネル数に対応<br>

* 最初の2層はストライド幅2となっており、ここで画像の幅と高さが1/4になる。
    * 計算負荷をさげるため目的
    * ただしフィルターサイズを大きくしているので大半の情報は維持される
* LRNはAlexNetと同様の目的で組込まれている
* 次の2層はボトルネック層として機能する（ボトルネック層は後述）
* モデルの中程にはインセプションモジュールを9個続けた深い層が続いている
* インセプションモジュールのスタックの途中では、データサイズを小さくして計算負荷を下げる目的で最大値プーリング層が組込まれている
* 最上位層では正則化のためのDropout→1000ラベルの確率を出力するための層が組込まれている

### インセプションモジュール

![](./fig/Inception_module.png)

* 入力を並列して複数のフィルターサイズのConv層に渡し、最後にチャンネル方向に結合している
* サイズの異なるフィルターを使うことで異なるパターンを検知できるようになった
* ただしフィルターを増やすとスパースなモデルとなり、パラメータの使い方が非効率になっていくため1×1Conv層をボトルネック層として使っている。<br>

### 1×1Conv層（ボトルネック層）の役割

* 1×1Conv層ではチャンネル方向の情報を集約している
* 集約による計算リソースの節約効果としては、下図の場合3×3×3=27個の要素の掛け算が必要だったが、(1×1×3)+(3×3)=12個の要素の掛け算で済む<br>
    ![](./fig/GoogLeNet_11conv.png)<br>
    図は https://ai-kenkyujo.com/artificial-intelligence/ai-architecture-02/ より
* 通常のConv層では空間パターン（楕円とか直線とか）を把握するが、この層では深度次元（チャンネル方向）のパターンを把握していると解釈できる

### 次元集約によるパラメータ効率化のイメージ
* フィルターを増やしても互いに無影響なノードが増えていくと、パラメータ数に比べてモデルの表現力が上がらない。（sparseなモデル）
* ノード間に相互作用がある層を作ることができれば効率的なモデルが出来る。
* チャンネル方向に集約してうまくいく直感的な理解としては<br>
    「（例えば）目なら同じような空間パターン（=図形）なので、『目に反応する層』として集約しても画像分類のための情報としてはそこまで減らないだろう」ということ
    ![](./fig/sparse_dense_layer_image.png)<br>
引用：[本質を捉えたデータ分析のための分析モデル入門 統計モデル、深層学習、強化学習等 用途・特徴から原理まで一気通貫! ](https://www.amazon.co.jp/dp/4802613776)

## ResNet

### 登場当時ＣＮＮアーキテクチャが抱えていた問題

**デグレーデーション（Degredation）問題**<br>
モデルを深くする方がかえって制度が落ちていく現象。<br>
テストデータに対する精度だけでなく、学習データに対する予測精度も低下してしまう。

数学的には層が深い層が精度が良くなるはず。（少なくとも学習データに対する予測はそうなるはず）<br>
層が深くなるほどモデルの表現力が高くなるからである。<br>

例えば20層もモデルと10層のモデルがあった場合、10層のモデルは20層のモデルに包含されるので、少なくとも10層と同等の精度は出るはずなのである。<br>
1案としては下位10層で10層のモデルを再現し、上位10層は恒等写像（入力をそのまま出力する）とすればよい。

しかし、実際にはそうはいかない。<br>
その原因は層が深くなりモデルが複雑になるほどパラメータの最適化が困難になっていくためと言われている。


これの問題を解消するために考案されたのがResNetである。<br>
その特徴を一言で言えば**恒等写像を学習のスタートラインにしたモデル**である。


### 特徴
* 残差ユニット(residual unit)を多数スタックする
    - スキップ接続（ショートカット接続）を使っていくつかの層の計算結果に下位の入力をそのまま加える。
    - これにより各残差ユニットは"（ターゲットとなる関数） - 入力値"を予測することになる。この学習を残差学習という。
    - 各層のパラメータは0に近い値で初期化するため、各残差ユニットは**恒等写像で初期化されている**と考えることもできる。<br>
    - 各残差ユニットではバッチ正則化、ReLU関数、3×3フィルター（ストライド幅1、sameパディング）2つのConv層を重ねている。（入力を足すためデータサイズは入力と同じにする）
    ![](./fig/residual_unit.png)

### アーキテクチャ
![](./fig/ResNet.png)

残差ユニットを数個通過するたびにフィルター数が倍になり、逆にフィルターの高さと幅が半分になる。<br>
これが起きる層では入力と出力のサイズが変わり入力をそのまま足すことが出来ないためストライド幅2のバッチ正則化を行ってサイズを合わせる。<br>
![](./fig/ResNet_resize_input.png)

シンプルなアイデアだが実際これで精度向上しているのがこのモデルのポイントである。

これはデグレーデーション問題が的を射ていたと考えられる。<br>

また、残差学習の方が学習効率が良いと考えられている。<br>
通常の学習では、0に近い値でパラメータを初期化するため、学習の序盤は多くのパラメータが0に近い値となり、逆伝播がモデルに行き渡るまでに時間がかかる。<br>
一方残差学習では学習序盤はほとんどが恒等写像であるため値を伝播させることはでき、結果として逆伝播がモデルに早く行き渡る。

## Xception

### 特徴
* Inceptionの発展型
* Inceptionで行ったこと（※）を極端にしたモデル<br>
    ※1×1Conv層で次元方向のパターン抽出、後続のConv層では空間方向のパターン抽出を行っており、両者を分離して学習している点
* パラメータ効率化がモデル導入の目的
* Inceptionモジュールを使う代わりに**深度分離可能な畳み込み層**（または**分離可能畳み込み層**）を使う

### Xception仮説
XceptionはXception仮説という考えに基づいて考案された。<br>
その説明は↓が分かりやすかったのでそのまま利用して説明する。<br>
[【論文読み】Xception : Deep Learning with Depthwise Separable Convolutions](https://qiita.com/woodyZootopia/items/3adc613e7717b6b5a260)<br>
これを読むと本と空間方向のパターンと次元方向のパターンの抽出順が逆な気もする

### パラメータ効率

■設定<br>
入力チャンネル : 128　<br>
出力チャンネル : 128　<br>
フィルターサイズ : 3×3 <br>
の層のパラメータ数


1．通常のConv層<br>
3×3×128×128 = 147,456

2．Xception<br>
(1×1×128×128)+(3×3×1×128) = 17,536<br>
※(1×1×128)→(3×3×1)を128セットと考える方が分かりやすいかも

※掛け算順はフィルター高さ×フィルター幅×チャンネル数（入力チャンネル数）×ノード数（出力チャンネル数）としている

→88.1%のパラメータ削減

パラメータ効率が凄まじく良いので（入力層などの）入力チャンネル数が少ない層でなければデフォルトで使用を検討したほうが良いらしい。

## SENet（Squeeze-and-Excitation Network）

## 特徴
* 直訳すると「圧縮と励起のネットワーク」
* 従来のモデルにSEブロックというサブネットワークを追加している
* SEブロックでは（名前の通り）情報を圧縮するステップと、重要な情報を強調するステップに分かれる。

## SEブロック
* SEブロックは既存のネットワークのレイヤの間に追加する。<br>
    ResNet、Inceptionに追加する場合以下のようになる。<br>
    ![](./fig/SENet_architecture.png)

* SEブロックの構造は以下である。
    - 【Squeezeステップ】グローバルプーリング層：チャンネルごとに平均をとる（これによって要素数がチャンネル数の1次元配列になる）
    - 【Excitationステップ】Dense層（ReLU）→Dense層（Sigmoid）：非線形変換&0~1変換
    - SEブロックの意味としては関係性が高いチャンネル（特徴）同士の活性化度合を高め、そうでないチャンネル（特徴）の活性化度合を緩める効果をもつと解釈できる（下図）<br>
    ![](./fig/SE_effect.png)

### SEブロックの考察

* 軽いため既存モデルに簡単に追加できる<br>
    - GAPでチャンネル方向の情報のみになるためSEブロックがもつパラメータ数は全体のネットワークに対して非常にすくない
    - 例えばチャンネル数が「入力256→中間層16→出力層256」の場合(256×16)+(16×256) = 8192個のパラメータとなる
    - 中間層は次元を小さくする。これにより特徴量の組み合わせの一般的な表現を学習する。
* どの深さのResNetに追加しても精度向上した<br>
    → 層を深くすることとは異なる精度向上を与えている（と考えられる）

⇒ デフォルトでモデルに追加することを検討してよさそう